In [1]:
import inspect
import re
import sys
from io import StringIO

from devtools import debug as _debug

In [2]:
import logging

# Set up logging
logging.basicConfig(level=logging.DEBUG)
logger = logging.getLogger(__name__)

In [3]:
class CustomDebug:
    """
    A wrapper class for the devtools debug function that provides custom output formatting.

    This class wraps the original debug function from the devtools library,
    modifying its output to remove unwanted information (such as ipykernel references)
    and preserve the original debug expressions, including cases with multiple arguments.

    Attributes:
        original_debug (function): The original debug function from devtools.

    Methods:
        __call__(*args, **kwargs): Invokes the debug function with custom output formatting.
        __getattr__(name): Provides access to attributes of the original debug object.

    Usage:
        debug = CustomDebug(_debug)
        debug(expression1, expression2, ...)  # Use like the original debug function
        with debug.timer('label'):  # Use timer and other methods as before
            ...

    The custom output removes ipykernel references and line numbers,
    and replaces argument placeholders with the actual debug expressions used in the code.
    This results in cleaner, more readable debug output, in Jupyter notebook-type environments.
    """
    def __init__(self, original_debug):
        self.original_debug = original_debug

    def __call__(self, *args, **kwargs):
        # logger.debug("CustomDebug called with args: %s, kwargs: %s", args, kwargs)

        frame = inspect.currentframe().f_back
        context = inspect.getframeinfo(frame).code_context
        if context:
            call_line = context[0].strip()
            debug_args = call_line[call_line.index('debug(') + 6:].rstrip(')').strip()
            # logger.debug("Extracted debug args: %s", debug_args)
        else:
            debug_args = 'unknown'
            # logger.debug("Could not extract debug args")

        arg_expressions = re.findall(r'(?:[^,()]|\([^()]*\))+', debug_args)
        arg_expressions = [arg.strip() for arg in arg_expressions]
        
        all_output = []
        for i, arg in enumerate(args):
            self.process_arg(arg, arg_expressions[i], all_output)

        for key, value in kwargs.items():
            self.process_arg(value, f"{key}={repr(value)}", all_output)

        cleaned_output = '\n'.join(all_output).strip()
        
        # logger.debug("Final cleaned output:\n%s", cleaned_output)
        print(cleaned_output)

    def process_arg(self, arg, expr, all_output):
        old_stdout = sys.stdout
        sys.stdout = StringIO()
        
        self.original_debug(arg)
        
        output = sys.stdout.getvalue()
        sys.stdout = old_stdout
        
        # logger.debug(f"Original debug output for {expr}:\n%s", output)
        
        lines = output.split('\n')
        cleaned_lines = []
        for line in lines:
            if 'ipykernel' not in line:
                line = re.sub(r'^.*?<module>\s*', '', line)
                # logger.debug("Processing line: %s", line)
                if line.strip().startswith('arg:'):
                    # Balance parentheses
                    open_parens = expr.count('(')
                    close_parens = expr.count(')')
                    if open_parens > close_parens:
                        expr += ')' * (open_parens - close_parens)
                    line = f"{expr}:{line.split(':', 1)[1]}"
                    logger.debug("Replaced arg placeholder: %s", line)
                cleaned_lines.append(line)
        
        all_output.extend(cleaned_lines)

    def __getattr__(self, name):
        return getattr(self.original_debug, name)

debug = CustomDebug(_debug)

## Example 1


Key differences and advantages of `debug()`:

- Formatting: `debug()` provides a well-indented, multi-line output that's much easier to read than the single-line output from print().
- Custom Classes: Both `print()` and `debug()` respect the custom `__repr__` of the `Employee` class, but `debug()` presents them in a more readable format.
- Complex Types: The `datetime.date` object and `numpy.array` are displayed clearly in the `debug()` output, making it easy to understand their types and values.
- Nested Structures: The nested dictionary (`revenue`) and list (`products`) are clearly formatted in the `debug()` output, improving readability.
- Context Information: debug() includes the file name and line number, which is extremely helpful when debugging larger code-bases.
- Colour Highlighting: In a terminal, `debug()` would also provide syntax highlighting, further enhancing readability (not shown in this text-based example).

In [4]:
import datetime
import numpy as np

In [5]:
class Employee:
    def __init__(self, name, role):
        self.name = name
        self.role = role

    def __repr__(self):
        return f"Employee('{self.name}', '{self.role}')"


company_data = {
    "name": "TechInnovate Inc.",
    "founded": datetime.date(2010, 5, 15),
    "employees": [
        Employee("Alice Johnson", "Software Engineer"),
        Employee("Bob Smith", "Data Scientist"),
    ],
    "revenue": {"2022": 1500000, "2023": 2000000},
    "products": ["AI Assistant", "Smart Analytics", "Cloud Services"],
    "market_share": np.array([0.05, 0.07, 0.06, 0.08]),
    "active": True,
}

In [6]:
print("Using print():\n")
print(company_data)

Using print():

{'name': 'TechInnovate Inc.', 'founded': datetime.date(2010, 5, 15), 'employees': [Employee('Alice Johnson', 'Software Engineer'), Employee('Bob Smith', 'Data Scientist')], 'revenue': {'2022': 1500000, '2023': 2000000}, 'products': ['AI Assistant', 'Smart Analytics', 'Cloud Services'], 'market_share': array([0.05, 0.07, 0.06, 0.08]), 'active': True}


In [7]:
print("\nUsing debug():\n")
debug(company_data)


Using debug():

company_data: {
        'name': 'TechInnovate Inc.',
        'founded': datetime.date(2010, 5, 15),
        'employees': [
            Employee('Alice Johnson', 'Software Engineer'),
            Employee('Bob Smith', 'Data Scientist'),
        ],
        'revenue': {
            '2022': 1500000,
            '2023': 2000000,
        },
        'products': [
            'AI Assistant',
            'Smart Analytics',
            'Cloud Services',
        ],
        'market_share': array([0.05, 0.07, 0.06, 0.08]),
        'active': True,
    } (dict) len=7


## Example 2

In [8]:
import time

In [9]:
x = [1, 2, 3]
debug(x)

x: [1, 2, 3] (list) len=3


In [10]:
def process_data(data):
    time.sleep(2.1)  # Simulating some processing time
    return [x * 2 for x in data]

In [11]:
large_dataset = list(range(1000))

with debug.timer("Data Processing"):
    result = process_data(large_dataset)

debug(len(result))

Data Processing: 2.104s elapsed
len(result): 1000 (int)


## Example 3:

Complex number example


In [12]:
class ComplexNumber:
    def __init__(self, real, imag):
        self.real = real
        self.imag = imag

    def __repr__(self):
        return f"ComplexNumber({self.real}, {self.imag})"


numbers = [ComplexNumber(1, 2), ComplexNumber(3, -4)]
debug(numbers)

numbers: [
        ComplexNumber(1, 2),
        ComplexNumber(3, -4),
    ] (list) len=2


## Examples from Documentation

Eg. A.

In [13]:
v1 = {
    'foo': {1: 'nested', 2: 'dict'},
    'bar': ['apple', 'banana', 'carrot', 'grapefruit'],
}

debug(v1, sum(range(5)))

v1: {
        'foo': {
            1: 'nested',
            2: 'dict',
        },
        'bar': [
            'apple',
            'banana',
            'carrot',
            'grapefruit',
        ],
    } (dict) len=2

sum(range(5)): 10 (int)


In [15]:
# Example 1: Single argument
x = [1, 2, 3]
debug(x)

# Example 2: Multiple arguments
v1 = {
    'foo': {1: 'nested', 2: 'dict'},
    'bar': ['apple', 'banana', 'carrot', 'grapefruit'],
}
debug(v1, sum(range(5)))

# Example 3: Function call
def square(n):
    return n ** 2
debug(square(5))

# Example 4: Complex expression
a, b = 10, 20
debug(a * b + len([1, 2, 3]))

# Example 5: With timer
large_dataset = list(range(1000))
with debug.timer('Data Processing'):
    result = [x**2 for x in large_dataset]
debug(len(result))

x: [1, 2, 3] (list) len=3
v1: {
        'foo': {
            1: 'nested',
            2: 'dict',
        },
        'bar': [
            'apple',
            'banana',
            'carrot',
            'grapefruit',
        ],
    } (dict) len=2

sum(range(5)): 10 (int)
square(5): 25 (int)
a * b + len([1): 203 (int)
Data Processing: 0.000s elapsed
len(result): 1000 (int)


In [16]:
foo = {
    'foo': np.array(range(20)),
    'bar': [{'a': i, 'b': {j for j in range(1 + i * 2)}} for i in range(3)],
    'spam': (i for i in ['i', 'am', 'a', 'generator']),
}

debug(foo)

# kwargs can be used as keys for what you are printing
debug(
    long_string='long strings get wrapped ' * 10,
    new_line_string='wraps also on newline\n' * 3,
)

bar = {1: 2, 11: 12}
# debug can also show the output of expressions
debug(
    len(foo),
    bar[1],
    foo == bar
)

foo: {
        'foo': (
            array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
                   17, 18, 19])
        ),
        'bar': [
            {
                'a': 0,
                'b': {0},
            },
            {
                'a': 1,
                'b': {0, 1, 2},
            },
            {
                'a': 2,
                'b': {
                    0,
                    1,
                    2,
                    3,
                    4,
                },
            },
        ],
        'spam': (
            'i',
            'am',
            'a',
            'generator',
        ),
    } (dict) len=3



IndexError: list index out of range